## Import Credentials

In [0]:
%run "./Config"

In [0]:
# Mounting Directory for Yelp JSONs Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/reviewin"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/reviewin has been unmounted.
Out[2]: True

In [0]:
from pyspark.sql.functions import udf,col

In [0]:
df = spark.read.format('json').load(mount_point+'/yelp_academic_dataset_review.json')

In [0]:
# Convert Date to type date
from pyspark.sql.functions import to_date, substring
df = df.withColumn('date',substring('date',1,10))
df =df.withColumn("date",to_date(df.date, "yyyy-MM-dd"))

In [0]:
from pyspark.sql import functions as F

df.select(*[
    (
        F.count(F.when((F.isnan(c) | F.col(c).isNull()), c)) if t not in ("timestamp", "date")
        else F.count(F.when(F.col(c).isNull(), c))
    ).alias(c)
    for c, t in df.dtypes if c in df.columns
]).show()

+-----------+----+----+-----+---------+-----+----+------+-------+
business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----------+----+----+-----+---------+-----+----+------+-------+

In [0]:
df.dtypes

Out[7]: [('business_id', 'string'),
 ('cool', 'bigint'),
 ('date', 'date'),
 ('funny', 'bigint'),
 ('review_id', 'string'),
 ('stars', 'double'),
 ('text', 'string'),
 ('useful', 'bigint'),
 ('user_id', 'string')]

In [0]:
df = df.withColumnRenamed('business_id', 'BusinessID')
df = df.withColumnRenamed('cool', 'Cool')
df = df.withColumnRenamed('date', 'Date')
df = df.withColumnRenamed('funny', 'Funny')
df = df.withColumnRenamed('review_id','ReviewID')
df = df.withColumnRenamed('stars','Stars')
df = df.withColumnRenamed('text', 'Text')
df = df.withColumnRenamed('useful', 'Useful')
df = df.withColumnRenamed('user_id', 'UserID')

In [0]:
bus = spark.read.format('csv').options(header='False').load(mount_point+'/business')

In [0]:
x = list(bus.select('_c1').toPandas()['_c1'])

In [0]:
df = df.filter(df.BusinessID.isin(x))

In [0]:
users = spark.read.format('json').load(mount_point+'/user_cleaned.json')

In [0]:
y = users.select('UserID')

In [0]:
df = df.join(y,on='UserID')

In [0]:
df.write.format('json').mode('overwrite').save(
    mount_point+'/reviews_cleaned.json')

In [0]:
# write to the Review table
try:
    df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
        .mode("append") \
        .option("dbtable", 'dbo.Review') \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')